# day 15

https://adventofcode.com/2019/day/15

In [ ]:
import logging
import logging.config
import os

import yaml

In [ ]:
with open('../logging.yaml') as fp:
    logging_config = yaml.load(fp, Loader=yaml.FullLoader)

logging.config.dictConfig(logging_config)

In [ ]:
FNAME = os.path.join('data', 'day15.txt')

LOGGER = logging.getLogger('day15')

## part 1

### problem statement:

#### loading data

In [ ]:
test_data = []

In [ ]:
def load_data(fname=FNAME):
    with open(fname) as fp:
        return [int(_) for _ in fp.read().strip().split(',')]

#### function def

In [ ]:
NORTH, SOUTH, WEST, EAST = range(1, 5)
DELTA = {NORTH: 1j,
         SOUTH: -1j,
         WEST: -1,
         EAST: 1, }

OPPO_DIRS = [[NORTH, SOUTH],
             [SOUTH, NORTH],
             [EAST, WEST],
             [WEST, EAST]]

HIT_WALL, STEPPED, AT_OXY = range(3)

In [ ]:
import aoc2019 as A

In [ ]:
from copy import deepcopy

def q_1(data):
    shell = {tuple(): A.IntcodeComputer(data, inputs=[])}
    seen_locs = set()
    while True:
        # for each position in the shell, take a step
        # then walk the robot through each path (needed
        # because we can't deep-copy the intcode prog)
        new_shell = {}
        for (step_sequence, ic) in shell.items():
            for step_dir in [NORTH, SOUTH, EAST, WEST]:
                # don't go backwards
                if step_sequence and (step_sequence[-1], step_dir) in OPPO_DIRS:
                    continue
                new_step_seq = step_sequence + (step_dir,)
                
                # convert this step sequence into a place on the grid
                # and if we've already been there, this is not the fastest
                # path so skip it
                curr_loc = sum(DELTA[_] for _ in new_step_seq)
                if curr_loc in seen_locs:
                    continue
                else:
                    seen_locs.add(curr_loc)
                
                ic_now = A.IntcodeComputer(intcode=ic.intcode,
                                           inputs=ic.inputs + [step_dir],
                                           inst_ptr=ic.inst_ptr,
                                           relative_base=ic.relative_base)
                curr_step = ic_now.get_output()
                if curr_step == AT_OXY:
                    return new_step_seq, seen_locs, curr_loc
                elif curr_step == STEPPED:
                    new_shell[new_step_seq] = ic_now
                else:
                    continue
        LOGGER.debug(new_shell)
        shell = new_shell

#### tests

In [ ]:
# def test_q_1():
#     LOGGER.setLevel(logging.DEBUG)
#     assert q_1(test_data) == True
#     LOGGER.setLevel(logging.INFO)

In [ ]:
# test_q_1()

#### answer

In [ ]:
new_step_seq, seen_locs, oxy_loc = q_1(load_data())
len(new_step_seq)

## part 2

### problem statement:

#### function def

In [ ]:
import networkx as nx

def q_2(data):
    g = nx.Graph()
    shell = {tuple(): A.IntcodeComputer(data, inputs=[])}
    seen_locs = set()
    while shell:
        # for each position in the shell, take a step
        # then walk the robot through each path (needed
        # because we can't deep-copy the intcode prog)
        new_shell = {}
        for (step_sequence, ic) in shell.items():
            for step_dir in [NORTH, SOUTH, EAST, WEST]:
                # don't go backwards
                if step_sequence and (step_sequence[-1], step_dir) in OPPO_DIRS:
                    continue
                new_step_seq = step_sequence + (step_dir,)
                
                ic_now = A.IntcodeComputer(intcode=ic.intcode,
                                           inputs=ic.inputs + [step_dir],
                                           inst_ptr=ic.inst_ptr,
                                           relative_base=ic.relative_base)
                curr_step = ic_now.get_output()
                
                if curr_step == HIT_WALL:
                    continue
                else:
                    prev_loc = sum(DELTA[_] for _ in step_sequence)
                    curr_loc = sum(DELTA[_] for _ in new_step_seq)
                    g.add_edge(prev_loc, curr_loc)
                    g.nodes[curr_loc]['at_oxy'] = curr_step == AT_OXY
                    
                    # add the new loc to the shell provided we haven't seen it before
                    if curr_loc not in seen_locs:
                        new_shell[new_step_seq] = ic_now
                
                seen_locs.add(curr_loc)
        LOGGER.debug(new_shell)
        shell = new_shell
        
    return max(nx.shortest_path_length(g, oxy_loc).values())

#### tests

In [ ]:
# def test_q_2():
#     LOGGER.setLevel(logging.DEBUG)
#     assert q_2(test_data) == True
#     LOGGER.setLevel(logging.INFO)

In [ ]:
# test_q_2()

#### answer

In [ ]:
LOGGER.setLevel(logging.INFO)
q_2(load_data())

fin